### 문장생성 구현

In [4]:
import numpy as np

a = np.array([1,2,3,4,5,6,7,8,9,10])
p = np.array([0.6,0.1,0.2,0.1,0,0,0,0,0,0])
sampled = np.random.choice(len(a), size=1, p=p) # 랜덤 초이스 -> 다양한 문장 생성
# sampled = np.array(np.argmax(p)).reshape(1) # 매번 0.6의 인덱스 0을 선택
print(a[sampled])

[3]


In [21]:
# coding: utf-8
import sys
sys.path.append('..')
import numpy as np
from common.functions import softmax
from ch06.rnnlm import Rnnlm
from ch06.better_rnnlm import BetterRnnlm

class MyRnnlmGen(Rnnlm): # 상속
    def generate(self, start_id, skip_ids=None, sample_size=100):
        word_ids = [start_id]

        x = start_id
        while len(word_ids) < sample_size:
            x = np.array(x).reshape(1, 1)
            score = self.predict(x)
#             print('score.shape=', score.shape)  # (1,1,10000) : 10000개의 voca 중 하나를 선택
#             print('score=', score) # softmax 타기 전까지의 값
#            break;
            
            
            p = softmax(score.flatten())        # (10000,)
#             print('p=',p)
#             print('len(p)=', len(p))
#             print('np.argmax(p)=', np.argmax(p))
#             print('np.max(p)=', np.max(p))
#             print(id_to_word[np.argmax(p)])
#             break
            

            sampled = np.random.choice(len(p), size=1, p=p)
#             sampled = np.array(np.argmax(p)).reshape(1)
            print('sampled=',sampled)
            print(id_to_word[np.argmax(p)][0])
            break
            
        
            if (skip_ids is None) or (sampled not in skip_ids): # <eos>와 같은 불용어는 건너뜀
                x = sampled
                word_ids.append(int(x))
#             print(word_ids)
        return word_ids

    def get_state(self):
        return self.lstm_layer.h, self.lstm_layer.c

    def set_state(self, state):
        self.lstm_layer.set_state(*state)


class BetterRnnlmGen(BetterRnnlm):
    def generate(self, start_id, skip_ids=None, sample_size=100):
        word_ids = [start_id]

        x = start_id
        while len(word_ids) < sample_size:
            x = np.array(x).reshape(1, 1)
            score = self.predict(x).flatten()
            p = softmax(score).flatten()

            sampled = np.random.choice(len(p), size=1, p=p)
            if (skip_ids is None) or (sampled not in skip_ids):
                x = sampled
                word_ids.append(int(x))

        return word_ids

    def get_state(self):
        states = []
        for layer in self.lstm_layers:
            states.append((layer.h, layer.c))
        return states

    def set_state(self, states):
        for layer, state in zip(self.lstm_layers, states):
            layer.set_state(*state)

### 문장생성을 위한 코드

In [22]:
# coding: utf-8
import sys
sys.path.append('..')
from rnnlm_gen import RnnlmGen
from dataset import ptb

corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
corpus_size = len(corpus)

model = MyRnnlmGen()
model.load_params('../ch06/Rnnlm.pkl')

# start 문자와 skip 문자 설정
start_word = 'you'
start_id = word_to_id[start_word]
print(start_id) # PTB에서 'you'd의 id 316
skip_words = ['N', '<unk>', '$']
skip_ids = [word_to_id[w] for w in skip_words]
print(skip_ids)

# 문장 생성
word_ids = model.generate(start_id, skip_ids, 100) # 문장
txt = ' '.join([id_to_word[i] for i in word_ids])  # 단어 사이의 공백 추가
txt = txt.replace(' <eos>', '.\n')
print(txt) # 확률분포 기반으로 매번 다르게 출력, 의미는 어색함

316
[27, 26, 416]
sampled= [9241]
<
you


### 더 좋은 문장으로

In [24]:
# coding: utf-8
import sys
sys.path.append('..')
from common.np import *
from rnnlm_gen import BetterRnnlmGen
from dataset import ptb


corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
corpus_size = len(corpus)


model = BetterRnnlmGen()
model.load_params('../ch06/BetterRnnlm.pkl')

# start 문자와 skip 문자 설정
# start_word = 'you'
# start_id = word_to_id[start_word]
# skip_words = ['N', '<unk>', '$']
# skip_ids = [word_to_id[w] for w in skip_words]
# # 문장 생성
# word_ids = model.generate(start_id, skip_ids)
# txt = ' '.join([id_to_word[i] for i in word_ids])
# txt = txt.replace(' <eos>', '.\n')

# print(txt)


model.reset_state()


# 'you' 일때
# start 문자와 skip 문자 설정
start_word = 'you'
start_id = word_to_id[start_word]
print(start_id) # PTB에서 'you'd의 id 316
skip_words = ['N', '<unk>', '$']
skip_ids = [word_to_id[w] for w in skip_words]
print(skip_ids)

# 문장 생성
word_ids = model.generate(start_id, skip_ids, 100) # 문장
txt = ' '.join([id_to_word[i] for i in word_ids])  # 단어 사이의 공백 추가
txt = txt.replace(' <eos>', '.\n')
print(txt) # 위에 RNNlm보다는 말이 된다..


# 문장 일때
# start_words = 'the meaning of life is'
# start_ids = [word_to_id[w] for w in start_words.split(' ')]
# print(start_ids)
# print(start_ids[:-1])

# for x in start_ids[:-1]:
#     x = np.array(x).reshape(1, 1)
#     model.predict(x)

# word_ids = model.generate(start_ids[-1], skip_ids)
# word_ids = start_ids[:-1] + word_ids
# txt = ' '.join([id_to_word[i] for i in word_ids])
# txt = txt.replace(' <eos>', '.\n')
# print('-' * 50)
# print(txt)


316
[27, 26, 416]
you see the rule and lawyers from this disaster or general merchandise administration sources said.
 the fact about one of several buyers 's ones growth says ronald of the federal reserve board 's integrity although that 's a base listing indicates that he feels a significant setback.
 he increase the chamber 's seventh improved stock upward than the previous year while mr. phelan 's resignation has been trading.
 the exchange under current will find rules not a attached to uncertainty with the real estate investment group the treasury considered sold at a discount to announce the sec
